# Sprint8 アンサンブル学習
---
3種類のアンサンブル学習をスクラッチ実装していきます。そして、それぞれの効果を小さめのデータセットで確認します。

-    ブレンディング
-    バギング
-    スタッキング

### 小さなデータセットの用意

以前も利用した回帰のデータセットを用意します。

House Prices: Advanced Regression Techniques

この中のtrain.csvをダウンロードし、目的変数としてSalePrice、説明変数として、GrLivAreaとYearBuiltを使います。

train.csvを学習データ8割、検証データ2割に分割してください。

In [24]:
import pandas as pd

df = pd.read_csv("train.csv")

X = df[['GrLivArea', 'YearBuilt']].values
y = df['SalePrice'].values

from sklearn.model_selection import train_test_split

# 訓練データと検証データの分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print(X_train.shape, X_test.shape, X.shape)

(1168, 2) (292, 2) (1460, 2)


## 【問題1】ブレンディングのスクラッチ実装
---
ブレンディング をスクラッチ実装し、単一モデルより精度があがる例を 最低3つ 示してください。精度があがるとは、検証データに対する平均二乗誤差（MSE）が小さくなることを指します。

### 単一モデルにおいて前処理、ハイパーパラメータなどの設定をしない場合

In [25]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error

# 線形回帰
lr = LinearRegression()
lr.fit(X_train, y_train)
lr_y_pred = lr.predict(X_test)
print("lr mse : ", mean_absolute_error(y_test, lr_y_pred))

# SVM
svm = SVR()
svm.fit(X_train, y_train)
svm_y_pred = svm.predict(X_test)
print("svm mse : ", mean_absolute_error(y_test, svm_y_pred))

# 決定木
cart = DecisionTreeRegressor()
cart.fit(X_train, y_train)
cart_y_pred = cart.predict(X_test)
print("cart mse : ", mean_absolute_error(y_test, cart_y_pred))

# Kmeans
knn = KNeighborsRegressor()
knn.fit(X_train, y_train)
knn_y_pred = knn.predict(X_test)
print("knn mse : ", mean_absolute_error(y_test, knn_y_pred))

# 各モデルに対して重み付けをして、足し合わせる。
blending_y_pred = 0.5 * lr_y_pred + \
                  0.1 * svm_y_pred + \
                  0.2 * cart_y_pred + \
                  0.2 * knn_y_pred
            
print("blending mse : ", mean_absolute_error(y_test, blending_y_pred))

ave_y_pred = (lr_y_pred + svm_y_pred + cart_y_pred + cart_y_pred)/4
print("ave mse : ", mean_absolute_error(y_test, ave_y_pred))

lr mse :  32711.07748550501
svm mse :  55442.18585049607
cart mse :  38523.850456621
knn mse :  34761.789041095886
blending mse :  30612.67605827354
ave mse :  33286.82526320544


### 単一モデルにおいて前処理、ハイパーパラメータなどの設定を行った場合→多様モデル

In [26]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import mean_absolute_error

# 標準化
scaler = StandardScaler()
scaler.fit(X_train) #trainデータのみFitを実行する
X_train_std = scaler.transform(X_train)
X_test_std = scaler.transform(X_test)

# PCA
pca = PCA(n_components=2)
pca.fit(X_train)
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

# 線形回帰
lr = LinearRegression()
lr.fit(X_train_std, y_train)
lr_y_pred = lr.predict(X_test_std)
print("lr mse : ", mean_absolute_error(y_test, lr_y_pred))

# SVM
svm = SVR(kernel='linear', C=10 )
svm.fit(X_train, y_train)
svm_y_pred = svm.predict(X_test)
print("svm mse : ", mean_absolute_error(y_test, svm_y_pred))

# 決定木
cart = DecisionTreeRegressor(random_state=0, max_depth=7)
cart.fit(X_train, y_train)
cart_y_pred = cart.predict(X_test)
print("cart mse : ", mean_absolute_error(y_test, cart_y_pred))

# Kmeans
knn = KNeighborsRegressor(n_neighbors=9)
knn.fit(X_train_std, y_train)
knn_y_pred = knn.predict(X_test_std)
print("knn mse : ", mean_absolute_error(y_test, knn_y_pred))

# 各モデルに対して重み付けをして、足し合わせる。
blending_y_pred = 0.05 * lr_y_pred + \
                  0.05 * svm_y_pred + \
                  0.3 * cart_y_pred + \
                  0.6 * cart_y_pred
            
print("blending mse : ", mean_absolute_error(y_test, blending_y_pred))

ave_y_pred = (lr_y_pred + svm_y_pred + cart_y_pred + cart_y_pred)/4
print("ave mse : ", mean_absolute_error(y_test, ave_y_pred))

lr mse :  32711.07748550502
svm mse :  31511.233790687565
cart mse :  27769.37648917789
knn mse :  26977.90525114155
blending mse :  27667.65696696641
ave mse :  28517.983694043774


## 考察
---
単一モデルよりもブレンディングしたことによってMSEの精度が高くなった。
適用した手法：線形回帰、SVM、決定木、Kmeansの４つの手法
前処理としては、データを標準化にし、線形回帰とKmeansモデルへ適用した。SVMと決定木は逆にMSEの値が悪くなったので、標準化しないデータで学習させた。
各モデルのハイパーパラメータをチューニングすることによって精度が向上したと言える。
特に、決定木とKmeansの２つに関してはかなり良い結果となった。

最後に各モデルの結果を見て、重み付けをすることによって、更に精度が高くなった。
良いモデルに関しては比較的重み付けの比重を多くした。

また、各モデルの予測値を平均化した場合でも、そこそこ精度が良かった。
ただ、ハイパーパラメータの設定によっては一部の単一モデルより精度が下がっている。

## 【問題2】バギングのスクラッチ実装
---
バギング をスクラッチ実装し、単一モデルより精度があがる例を 最低1つ 示してください。

バギングとは

バギングは入力データの選び方を多様化する方法です。訓練データから重複を許した上でランダムに抜き出すことで、N種類のサブセット（ ブートストラップサンプル ）を作り出します。それらによってモデルをN個学習し、推定結果の平均をとります。ブレンディングと異なり、それぞれの重み付けを変えることはありません。

sklearn.model_selection.train_test_split — scikit-learn 0.21.3 documentation

推定結果の平均をとる部分はブレンディングと同様の実装になります。

In [27]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

df = pd.read_csv("train.csv")

models = [
            ['lr', LinearRegression()],
            ['svm', SVR(kernel='linear', C=10 )],
            ['tree', DecisionTreeRegressor(random_state=0, max_depth=7)],
            ['knn', KNeighborsRegressor(n_neighbors=9)]
        ]

y_pred_list = []
y_test_list = []
for model_name, model in models:
    
    # 全体の6割をサンプリングする
    data = df.sample(frac=0.6, replace=True)

    X = data[['GrLivArea', 'YearBuilt']].values
    y = data['SalePrice'].values

    # 訓練データと検証データの分割
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_pred_list.append(y_pred)
    y_test_list.append(y_test)
    print("{} mse : {}".format(model_name, mean_absolute_error(y_test, y_pred)))

ave_y_test = np.array(y_test_list).mean(axis=0)
ave_y_pred = np.array(y_pred_list).mean(axis=0)
print("Average mse : ", mean_absolute_error(ave_y_test, ave_y_pred))

for i, test in enumerate(y_test_list):
    print("Sample{}, mse : {}".format(i+1, mean_absolute_error(test, ave_y_pred)))

lr mse : 32141.995843820063
svm mse : 28304.659350887352
tree mse : 26090.310849895643
knn mse : 26864.505050505057
Average mse :  15414.124428038422
Sample1, mse : 50874.54448187247
Sample2, mse : 52567.54183042638
Sample3, mse : 49423.76511073222
Sample4, mse : 45714.730972450074


## 考察
---
各モデルで使用したy_testデータでMSEを算出した結果、単一モデルの方が良い結果となった。
これは想定外の結果ですが、何かバギングのやり方が間違っているのでしょうか？

また、各モデルで使用したy_testの平均値のMSEの値は良いが、そもそもy_testで使用するデータを平均するのはおかしいと思うのですが、どうなのでしょうか？

## 【問題3】スタッキングのスクラッチ実装
---
スタッキング をスクラッチ実装し、単一モデルより精度があがる例を 最低1つ 示してください。

In [28]:
import pandas as pd

df = pd.read_csv("train.csv")

X = df[['GrLivArea', 'YearBuilt']].values
y = df['SalePrice'].values

from sklearn.model_selection import train_test_split

# 訓練データと検証データの分割
X_train_org, X_test, y_train_org, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [29]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold

models = [
            ['lr', LinearRegression()],
            ['svm', SVR()],
            ['cart', DecisionTreeRegressor()],
            ['knn', KNeighborsRegressor()]
        ]


# クロスバリデーション
n_folds = 5
kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

blend_train = np.zeros((X_train_org.shape[0], len(models)))
blend_test = np.zeros((X_test.shape[0], len(models)))

for j, (name, model) in enumerate(models):
    
    blend_test_j = np.zeros((X_test.shape[0], n_folds))

    # データセットを分割
    for i, (train_idx, valid_idx) in enumerate(kf.split(X_train_org, y_train_org)):
        
        X_train, y_train = X_train_org[train_idx], y_train_org[train_idx]
        X_valid, y_valid = X_train_org[valid_idx], y_train_org[valid_idx]
        
        model.fit(X_train, y_train)
        blend_train[valid_idx, j] = model.predict(X_valid)
        blend_test_j[:, i] = model.predict(X_test)

    # クロスバリデーションした予測値を平均化
    blend_test[:, j] = blend_test_j.mean(axis=1)
    print("{} mse: {}".format(name, mean_absolute_error(y_test, blend_test_j.mean(axis=1))))


blender = LinearRegression()
blender.fit(blend_train, y_train_org)
blender_pred = blender.predict(blend_test)

print("blending mse : ", mean_absolute_error(y_test, blender_pred))


lr mse: 32710.256655269444
svm mse: 55464.79206621753
cart mse: 30528.340410958906
knn mse: 33322.31534246575
blending mse :  30111.604666393647


## 考察
---
スタッキングで実行した結果、MSEの精度は上がった。

ただ、各モデルのハイパーパラメータを設定すると、以下のように一部の単一モデル（cart）より精度が落ちる場合がある。
これは、起こり得ることなのでしょうか？

lr mse: 32710.256655269444  
svm mse: 31514.52887818637   
cart mse: 25469.247447367132  
knn mse: 32499.18325722983   
blending mse :  28163.498199440211

In [30]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold

models = [
            ['lr', LinearRegression()],
            ['svm', SVR(kernel='linear', C=10 )],
            ['cart', DecisionTreeRegressor(random_state=0, max_depth=7)],
            ['knn', KNeighborsRegressor(n_neighbors=9)]
        ]


# クロスバリデーション
n_folds = 5
kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

blend_train = np.zeros((X_train_org.shape[0], len(models)))
blend_test = np.zeros((X_test.shape[0], len(models)))

for j, (name, model) in enumerate(models):
    
    blend_test_j = np.zeros((X_test.shape[0], n_folds))

    # データセットを分割
    for i, (train_idx, valid_idx) in enumerate(kf.split(X_train_org, y_train_org)):
        
        X_train, y_train = X_train_org[train_idx], y_train_org[train_idx]
        X_valid, y_valid = X_train_org[valid_idx], y_train_org[valid_idx]
        
        model.fit(X_train, y_train)
        blend_train[valid_idx, j] = model.predict(X_valid)
        blend_test_j[:, i] = model.predict(X_test)

    # クロスバリデーションした予測値を平均化
    blend_test[:, j] = blend_test_j.mean(axis=1)
    print("{} mse: {}".format(name, mean_absolute_error(y_test, blend_test_j.mean(axis=1))))


blender = LinearRegression()
blender.fit(blend_train, y_train_org)
blender_pred = blender.predict(blend_test)

print("blending mse : ", mean_absolute_error(y_test, blender_pred))


lr mse: 32710.256655269444
svm mse: 31514.52887818637
cart mse: 25469.247447367132
knn mse: 32499.18325722983
blending mse :  28163.49819944021
